In [3]:
!pip install biopython
import requests
from Bio import SeqIO
from time import sleep
from google.colab import files

uploaded = files.upload()

fasta_file = list(uploaded.keys())[0]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.4 MB/s eta 0:00:00


Saving combined_filtered_filtered_95 (1) (2)_cleaned.fasta to combined_filtered_filtered_95 (1) (2)_cleaned (1).fasta


In [8]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO

# Function to perform BLASTP search and print top 5 hits
def blastp_and_print_hits(seq_record):
    print("Query ID:", seq_record.id)
    result_handle = NCBIWWW.qblast("blastp", "nr", seq_record.seq)
    blast_records = NCBIXML.parse(result_handle)
    for i, blast_record in enumerate(blast_records):
        if i >= 5:  # Print only the top 5 hits
            break
        for alignment in blast_record.alignments[:5]:  # Limit to top 5 hits
            print("Hit:", alignment.title)
    result_handle.close()

# Upload protein FASTA file
from google.colab import files
uploaded = files.upload()

# Iterate over each sequence in the uploaded FASTA file
for filename in uploaded.keys():
    with open(filename, "r") as fasta_file:
        for seq_record in SeqIO.parse(fasta_file, "fasta"):
            blastp_and_print_hits(seq_record)



Saving combined_filtered_filtered_95 (1) (2)_cleaned.fasta to combined_filtered_filtered_95 (1) (2)_cleaned (4).fasta
Query ID: EPO_JA641346_Sequence_2_from_Patent_EP2385982.


KeyboardInterrupt: 

In [ ]:
import requests
from Bio import SeqIO
from time import sleep
from xml.etree import ElementTree as ET

# Function to run BLASTP using the NCBI BLAST API
def run_blastp(sequence):
    url = 'https://blast.ncbi.nlm.nih.gov/Blast.cgi'
    params = {
        'CMD': 'Put',
        'DATABASE': 'nr',
        'PROGRAM': 'blastp',
        'QUERY': sequence,
        'FORMAT_TYPE': 'XML'
    }
    response = requests.post(url, data=params)
    response.raise_for_status()
    rid = None
    for line in response.text.splitlines():
        if 'RID =' in line:
            rid = line.split('=')[1].strip()
            break
    if not rid:
        raise Exception("Failed to obtain RID")

    # Wait for the result to be ready
    sleep(10)

    params = {
        'CMD': 'Get',
        'RID': rid,
        'FORMAT_TYPE': 'XML'
    }
    result_response = requests.get(url, params=params)
    result_response.raise_for_status()
    return result_response.text

# Function to parse BLASTP results and extract the top 5 hits
def parse_blast_results(xml_result):
    root = ET.fromstring(xml_result)
    hits = []
    for iteration in root.findall(".//Iteration"):
        for hit in iteration.findall(".//Hit")[:5]:
            hit_id = hit.find(".//Hit_accession").text
            hit_def = hit.find(".//Hit_def").text
            hit_score = hit.find(".//Hsp_score").text
            hit_evalue = hit.find(".//Hsp_evalue").text
            hit_link = f"https://www.ncbi.nlm.nih.gov/protein/{hit_id}"
            hits.append((hit_id, hit_def, hit_score, hit_evalue, hit_link))
    return hits

# Read the FASTA file and run BLASTP for each sequence
sequences = SeqIO.parse(fasta_file, "fasta")
for seq_record in sequences:
    print(f"Processing {seq_record.id}...")
    blast_result = run_blastp(str(seq_record.seq))
    top_hits = parse_blast_results(blast_result)

    print(f"Top 5 hits for {seq_record.id}:")
    for hit in top_hits:
        print(f"ID: {hit[0]}, Definition: {hit[1]}, Score: {hit[2]}, E-value: {hit[3]}")
        print(f"Link: {hit[4]}")
    print("\n")
    sleep(1)  # To avoid overloading the server
